In [64]:
import pandas as pd
import matplotlib
import plotly
import tkinter
import os
import numpy as np
import seaborn
import plotly.express
import kaleido

In [65]:
with open('teams_path') as f:
    lines = f.readlines()
teams_path = lines[0] + '/'
print(teams_path)

C:\Teams\Hoogheemraadschap van Delfland\Programma DigiTrans - Zoutindringing/


In [66]:
save_TS_plots = True
save_violin_plots = True
save_quantile_plots = False
locatie = 'schiegemaal'
#pd.options.plotting.backend = "plotly"
pd.options.plotting.backend = "matplotlib"

In [67]:

df = pd.read_table(teams_path + 'telecontrol/'+locatie+'.csv',delimiter = '\t')
numeric_cols = df.columns[2:]
df[numeric_cols] = df[numeric_cols].stack().str.replace(',','.').unstack()
df.head()
df.drop(df.tail(5).index,inplace = True)
df['datetime'] = pd.to_datetime(df['Datum'] + ' ' + df['Tijd (Europe/Amsterdam)'])
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

#df['OW000-008/WNS7670 - gemeten waarde [mS/cm]'] = pd.to_numeric(df['OW000-008/WNS7670 - gemeten waarde [mS/cm]'],errors='coerce')
df.head()
df = df.set_index('Datum')
df[numeric_cols] = np.where(df[numeric_cols]>=100 , np.nan, df[numeric_cols])
df['maand'] = df['datetime'].dt.strftime('%b')


In [69]:
%%capture
isExist = os.path.exists(teams_path+'plots/' + locatie)
if not isExist:
  os.makedirs(teams_path+'plots/' + locatie)


if save_TS_plots:
  root = tkinter.Tk()
  my_dpi = root.winfo_fpixels('1i')

  subsets =  (['2015-01-01','2015-31-12'], ['2016-01-01','2016-31-12'], ['2017-01-01','2017-31-12'], ['2018-01-01','2018-31-12'],
              ['2019-01-01','2019-31-12'], ['2020-01-01','2020-31-12'], ['2021-01-01','2021-31-12'], ['2022-01-01','2022-31-12'],
              ['2015-01-01','2022-31-12']
              )


  for subset in subsets:
  #subset = ['2015-01-01','2015-31-12']
      plot1 = df.loc[subset[0]:subset[1]].plot('datetime', df.columns[2],figsize=(1920/my_dpi, 1080/my_dpi))
      axes= plot1.xaxis
      axes.set_major_locator(matplotlib.dates.MonthLocator(interval=1))
      axes.get_major_ticks()[-1].label1.set_visible(False)
      plot1.set_xlabel('Datum')
      plot1.set_ylabel('EGV')
      plot1.set_title('Van ' + subset[0] + ' t/m ' + subset[1] + '\n Locatie: ' + locatie)
      plot = plot1.figure
      plot_int = plotly.tools.mpl_to_plotly(plot)
      plot_int.write_html(teams_path+"plots/" + locatie + '/' + locatie + subset[0] + '_' + subset[1] + ".html")
      plotly.offline.iplot(plot_int,filename= teams_path+"plots/" + locatie + '/' + locatie + subset[0] + '_' + subset[1] + ".png")


if save_violin_plots:
  plot2 = plotly.express.violin(df,x = 'maand', y = numeric_cols[1],points=False,title='EGV verdeling per maand op locatie: ' + locatie, labels= {numeric_cols[1] : 'EGV (mS/cm)'},
  category_orders={"maand": ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]})
  #plot2.set_ylabel('EGV (mS/cm')
  #plot2.show()
  #plot2 = plot2.figure
  #plot_int = plotly.tools.mpl_to_plotly(plot2)
  plot2.write_html(teams_path+"plots/" + locatie + '/' + locatie + "_violin.html")
  plotly.offline.iplot(plot_int,filename= teams_path+"plots/" + locatie + '/' + locatie + "_violin.png")
#if save_quantile_plots:
  

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:

df.groupby('maand').quantile([0.10,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90])

C:\Users\ds_sboeschoten\AppData\Local\Temp\3\ipykernel_7676\3088433796.py:1: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.



OW000-014/WNS3838 - gemeten waarde [°C]  \
maand                                                
Apr   0.1                                      7.9   
      0.2                                      8.4   
      0.3                                      9.3   
      0.4                                     10.3   
      0.5                                     11.4   
...                                            ...   
Sep   0.5                                     17.9   
      0.6                                     18.2   
      0.7                                     18.4   
      0.8                                     18.8   
      0.9                                     19.6   

           OW000-014/WNS7670 - gemeten waarde [mS/cm]            datetime  
maand                                                                      
Apr   0.1                                      1.2580 2017-04-25 14:23:06  
      0.2                                      1.4520 2018-04-10 16:14:00  
      0.3                                      1.5340 2018-04-25 19:12:18  
      0.4                                      1.6800 2019-04-29 11:50:24  
      0.5                                      1.9050 2020-04-13 22:05:30  
...                                               ...                 ...  
Sep   0.5                                      4.2000 2019-09-10 12:32:00  
      0.6                                      4.5940 2019-09-24 05:09:00  
      0.7                                      4.9850 2020-09-07 17:20:30  
      0.8                                      5.5710 2020-09-21 09:32:00  
      0.9                                      6.3223 2021-09-04 21:24:30  

[108 rows x 3 columns]